In [2]:
## Following MATLAB code from http://capture-clarity.org/clarity-based-tractography/

In [4]:
import os

In [5]:
## Parameters (the script loops through all parameters and saves each result automatically)
dogsigmaArr = [1];  # Sigma values for derivative of gaussian filter, recommended value: 0.6 - 1.3 (based on actual data)
gausigmaArr = [2.3];  # Sigma values for gaussian filter, recommended value: 1.3 - 2.3 (based on actual data)
angleArr = [25];  # Angle thresholds for fiber tracking, recommended value: 20 - 30

In [6]:
pwd

u'/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb'

In [7]:
import numpy as np

In [8]:
import math
from scipy import ndimage
import nibabel as nib

In [9]:
## Change later on
file_path = "/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/TIFF_stack"
directory = os.path.dirname(file_path)

In [10]:
cd TIFF_stack/

/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/TIFF_stack


In [112]:
from PIL import Image
from numpy import matlib
from scipy import signal

In [142]:
# Set up results directory
if not os.path.exists(directory):
    os.makedirs(directory)

# im = Image.open('page1.tiff')  # Needs to be changed to dynamically go down list of fnDataArr (currently just loads same test image)
# Omitted: channel data (red/green - our CLARITY data was single channel, so no channel data loaded.)
img_data = np.single(100 * np.ones((10, 10, 10)));  #data is hard coded to be np.ones

print img_data[0][0][0];

for jj in range(len(dogsigmaArr)):
    dogsigma = dogsigmaArr[jj];
    print "Start DoG Sigma on " + str(dogsigma);

    # Generate dog kernels
    dogkercc = doggen([dogsigma, dogsigma, dogsigma]);
    dogkercc = np.transpose(dogkercc1, (0, 1, 2));  # annoying

    #print dogkercc.shape;
    #print dogkercc[:, :, 0];
    
    dogkerrr = np.transpose(dogkercc, (1, 0, 2));
    
    #print dogkerrr[:, :, 0];
    dogkerzz = np.transpose(dogkercc, (0, 2, 1));

    #print dogkerzz[:, :, 0];
    
    # Compute gradients
    grr = signal.fftconvolve(img_data, dogkercc, 'same');
    grr = np.transpose(grr, (1, 0, 2));
    
    #print grr[:, :, 0];
    
    gcc = signal.fftconvolve(img_data, dogkerrr, 'same');
    gcc = np.transpose(gcc, (1, 0, 2));
    
    #print gcc[:, :, 0];
    
    gzz = signal.fftconvolve(img_data, dogkerzz, 'same');
    gzz = np.transpose(gzz, (1, 0, 2));
    
    #print gzz[:, :, 0];
    
    # Compute gradient products
    gprrrr = np.multiply(grr, grr);
    
    #print gprrrr[:, :, 0];
    
    gprrcc = np.multiply(grr, gcc);
    
    #print gprrcc[:, :, 0];
    
    gprrzz = np.multiply(grr, gzz);
    
    #print gprrzz[:, :, 0]
    
    gpcccc = np.multiply(gcc, gcc);
    gpcczz = np.multiply(gcc, gzz);
    gpzzzz = np.multiply(gzz, gzz);

    # Compute gradient amplitudes
    #print ga.dtype;
    ga = np.sqrt(gprrrr + gpcccc + gpzzzz);
    
    #print ga[:, :, 0];
    
    print "GA SHAPE:"
    print ga.shape;

    # Convert numpy ndarray object to Nifti data type
    gradient_amplitudes_data = nib.Nifti1Image(ga, affine=np.eye(4));

    # Save gradient amplitudes image 
    nib.save(gradient_amplitudes_data, 'gradient_amplitudes.nii');

    # Compute gradient vectors
    gv = np.concatenate((grr[..., np.newaxis], gcc[..., np.newaxis], gzz[..., np.newaxis]), axis = 3);
    gv = np.divide(gv, np.tile(ga[..., None], [1, 1, 1, 3]));
    #print gv[:, :, 0, 1];
    
    print "GV SHAPE:"
    print gv.shape;
    
    # Convert numpy ndarray object to Nifti data type
    gradient_vectors_data = nib.Nifti1Image(gv, affine=np.eye(4));

    # Save gradient vectors
    nib.save(gradient_vectors_data, 'gradient_vectors.nii');

    # Compute structure tensor
    for kk in range(len(gausigmaArr)):
        gausigma = gausigmaArr[kk];
        print "Start Gauss Sigma with gausigma = " + str(gausigma);

        print "Generating Gaussian kernel..."
        gaussker = np.single(gaussgen([gausigma, gausigma, gausigma]));
        
        #print gaussker[:, :, 0];

        print "Blurring gradient products..."
        gprrrrgauss = signal.fftconvolve(gprrrr, gaussker, "same");
        gprrccgauss = signal.fftconvolve(gprrcc, gaussker, "same");
        gprrzzgauss = signal.fftconvolve(gprrzz, gaussker, "same");
        gpccccgauss = signal.fftconvolve(gpcccc, gaussker, "same");
        gpcczzgauss = signal.fftconvolve(gpcczz, gaussker, "same");
        gpzzzzgauss = signal.fftconvolve(gpzzzz, gaussker, "same");

        print "Saving a copy for this Gaussian sigma..."
        tensorfsl = np.concatenate((gprrrrgauss[..., np.newaxis], gprrccgauss[..., np.newaxis], gprrzzgauss[..., np.newaxis], gpccccgauss[..., np.newaxis], gpcczzgauss[..., np.newaxis], gpzzzzgauss[..., np.newaxis]), axis = 3);

         # Convert numpy ndarray object to Nifti data type
        tensor_fsl_data = nib.Nifti1Image(tensorfsl, affine=np.eye(4));

        nib.save(tensor_fsl_data, "dogsigma_" + str(jj) + "gausigma_" + str(kk) + 'tensorfsl.nii');
            
print 'Complete!'

100.0
Start DoG Sigma on 1
GA SHAPE:
(10, 10, 10)
GV SHAPE:
(10, 10, 10, 3)
Start Gauss Sigma with gausigma = 2.3
Generating Gaussian kernel...
Blurring gradient products...
Saving a copy for this Gaussian sigma...
Complete!


In [75]:
'''
Function to generate derivatives of Gaussian kernels, in either 1D, 2D, or 3D.
Source code in MATLAB obtained from Qiyuan Tian, Stanford University, September 2015
Edited to work in Python by Tony
'''

def doggen(sigma):
    halfsize = np.ceil(3 * np.max(sigma))
    x = range(np.single(-halfsize), np.single(halfsize + 1));  # Python colon is not inclusive at end, while MATLAB is.
    dim = len(sigma);
    
    if dim == 1:
        X = np.array(x);  # Remember that, by default, numpy arrays are elementwise multiplicative
        k = -X * np.exp(-X**2/(2 * sigma**2));
        
    elif dim == 2:
        [X, Y] = np.meshgrid(x, x);
        k = -X * np.exp(-X**2/(2*sigma[0]^2) * np.exp(-Y**2))
        
    elif dim == 3:
        [X, Y, Z] = np.meshgrid(x, x, x);
        X = X.transpose(0, 2, 1);  # Obtained through vigorous testing (see below...)
        Y = Y.transpose(2, 0, 1);
        Z = Z.transpose(2, 1, 0);
        
        X = X.astype(float);
        Y = Y.astype(float);
        Z = Z.astype(float);
        k = -X * np.exp(np.divide(-np.power(X, 2), 2 * np.power(sigma[0], 2))) * np.exp(np.divide(-np.power(Y,2), 2 * np.power(sigma[1],2))) * np.exp(np.divide(-np.power(Z,2), 2 * np.power(sigma[2],2)))
        
    else:
        print 'Only supports up to 3 dimensions'
        
    return np.divide(k, np.sum(np.abs(k[:])));

In [77]:
'''
Function to generate Gaussian kernels, in 1D, 2D and 3D.
Source code in MATLAB obtained from Qiyuan Tian, Stanford University, September 2015
Edited to work in Python by Tony. 
'''

def gaussgen(sigma):
    halfsize = np.ceil(3 * max(sigma));
    x = range(np.single(-halfsize), np.single(halfsize + 1));

    dim = len(sigma);

    if dim == 1:
        k = np.exp(-x**2 / (2 * sigma^2));
    
    elif dim == 2:
        [X, Y] = np.meshgrid(x, x);
        k = np.exp(-X**2 / (2 * sigma[0]**2)) * np.exp(-Y**2 / (2 * sigma[1]**2)); 
    
    elif dim == 3:
        [X, Y, Z] = np.meshgrid(x, x, x);
        X = X.transpose(0, 2, 1);  # Obtained through vigorous testing (see below...)
        Y = Y.transpose(2, 0, 1);
        Z = Z.transpose(2, 1, 0);
        
        X = X.astype(float);  # WHY PYTHON?
        Y = Y.astype(float);
        Z = Z.astype(float);
        k = np.exp(-X**2 / (2 * sigma[0]**2)) * np.exp(-Y**2 / (2 * sigma[1]**2)) * np.exp(-Z**2 / (2 * sigma[2]**2));
    
    else:
        print 'Only supports up to dimension 3'

    return np.divide(k, np.sum(np.abs(k)));

In [143]:
## Compare tensor_fsl_data with imported values from MATLAB
print tensorfsl.shape  # numpy

(10, 10, 10, 6)


In [80]:
MATLAB_output = nib.load("/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/test_MATLAB_10by10by10_outputs/dogsig1_gausig2.3/test_MATLAB_tensorfsl_dogsig1_gausig2.3.nii")

In [81]:
print MATLAB_output.shape

(10, 10, 10, 6)


In [97]:
MATLAB_np_array = MATLAB_output.get_data()
print MATLAB_np_array.shape

(10, 10, 10, 6)


In [146]:
print tensorfsl[:, :, :, 0];

[[[ 109.85943033  147.37287499  179.26563687  201.37589584  212.3919397
    212.3919397   201.37589584  179.26563687  147.37287499  109.85943033]
  [ 147.37287499  197.69594847  240.47905765  270.13925545  284.91692228
    284.91692228  270.13925545  240.47905765  197.69594847  147.37287499]
  [ 179.26563687  240.47905765  292.52080083  328.5997214   346.57540288
    346.57540288  328.5997214   292.52080083  240.47905765  179.26563687]
  [ 201.37589584  270.13925545  328.5997214   369.12854264  389.32130808
    389.32130808  369.12854264  328.5997214   270.13925545  201.37589584]
  [ 212.3919397   284.91692228  346.57540288  389.32130808  410.61869621
    410.61869621  389.32130808  346.57540288  284.91692228  212.3919397 ]
  [ 212.3919397   284.91692228  346.57540288  389.32130808  410.61869621
    410.61869621  389.32130808  346.57540288  284.91692228  212.3919397 ]
  [ 201.37589584  270.13925545  328.5997214   369.12854264  389.32130808
    389.32130808  369.12854264  328.5997214   

In [147]:
print MATLAB_np_array[:, :, :, 0];

[[[ 109.85939789  147.37295532  179.26557922  201.37585449  212.39183044
    212.39196777  201.37599182  179.26565552  147.37287903  109.85943604]
  [ 147.37289429  197.69589233  240.47901917  270.13919067  284.91690063
    284.91680908  270.13919067  240.47918701  197.69593811  147.37290955]
  [ 179.26560974  240.47901917  292.52096558  328.59942627  346.57531738
    346.57531738  328.59970093  292.52087402  240.47904968  179.26560974]
  [ 201.37588501  270.13912964  328.59945679  369.12838745  389.32110596
    389.32110596  369.12866211  328.59973145  270.13916016  201.37582397]
  [ 212.39186096  284.91671753  346.57528687  389.3210144   410.61871338
    410.61868286  389.32144165  346.57556152  284.91680908  212.391922  ]
  [ 212.39189148  284.91677856  346.57519531  389.32107544  410.6187439
    410.61865234  389.32144165  346.57556152  284.9168396   212.39190674]
  [ 201.37590027  270.13916016  328.59954834  369.12850952  389.3213501
    389.32125854  369.12860107  328.59979248  2

In [162]:
truth_boolean = np.isclose(tensorfsl, MATLAB_np_array)
print truth_boolean;

[[[[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  ..., 
  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ Tr

In [102]:
dogkercc1 = doggen([dogsigma, dogsigma, dogsigma]);
dogkercc1 = np.transpose(dogkercc1, (1, 2, 0))
print dogkercc1[:, :, 0];


dogkerrr1 = np.transpose(dogkercc1, (1, 0, 2));
#print dogkerrr1[:, :, 0];


[[  3.59647946e-07   2.92093928e-06   6.54537083e-06  -0.00000000e+00
   -6.54537083e-06  -2.92093928e-06  -3.59647946e-07]
 [  4.38140893e-06   3.55843252e-05   7.97389406e-05  -0.00000000e+00
   -7.97389406e-05  -3.55843252e-05  -4.38140893e-06]
 [  1.96361125e-05   1.59477881e-04   3.57365139e-04  -0.00000000e+00
   -3.57365139e-04  -1.59477881e-04  -1.96361125e-05]
 [  3.23744763e-05   2.62934575e-04   5.89195506e-04  -0.00000000e+00
   -5.89195506e-04  -2.62934575e-04  -3.23744763e-05]
 [  1.96361125e-05   1.59477881e-04   3.57365139e-04  -0.00000000e+00
   -3.57365139e-04  -1.59477881e-04  -1.96361125e-05]
 [  4.38140893e-06   3.55843252e-05   7.97389406e-05  -0.00000000e+00
   -7.97389406e-05  -3.55843252e-05  -4.38140893e-06]
 [  3.59647946e-07   2.92093928e-06   6.54537083e-06  -0.00000000e+00
   -6.54537083e-06  -2.92093928e-06  -3.59647946e-07]]


In [74]:
sigma = [1, 1, 1]
halfsize = 3;
x = range(np.single(-halfsize), np.single(halfsize + 1));
[X, Y, Z] = np.meshgrid(x, x, x);
X = X.transpose(0, 2, 1);
Y = Y.transpose(2, 0, 1);
Z = Z.transpose(2, 1, 0);

print X.dtype;
X = X.astype(float);
Y = Y.astype(float);
Z = Z.astype(float);
print X.dtype;

print (-X **2) / (2 * np.power(sigma[0], 2))

print -X * np.exp(np.divide(-X**2, 2 * np.power(sigma[0], 2)));

int64
float64
[[[-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]]

 [[-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]]

 [[-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]]

 [[-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4.5 -2.  -0.5 -0.  -0.5 -2.  -4.5]
  [-4